In [ ]:
#devtools::install_github(repo="knausb/vcfR")
#install.packages('doParallel')
library(vcfR)
library(doParallel)
setwd("/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/")
dir.create("Analysis")

# VCF-------------------
vcfDir <- "ensemble/vcf/"
filenames <- dir(vcfDir,"ensemble.snpEff.vcf.gz")
#filenames <- filenames[!grepl(".tbi",filenames)]

vcfRead <- function(filename,vcfDir){
  tumorID <- sub("-ensemble.snpEff.vcf.gz","",filename)
  tmp <- vcfR::read.vcfR(paste0(vcfDir,filename), verbose=FALSE)
  tmpVcf <- tmp@fix
  tmpGt <- tmp@gt
  if(length(colnames(tmpGt))==3){
    if(!grepl("_N\\b",colnames(tmpGt)[3])){ #CHECK THE NAME OF TUMOR AND NORMAL SAMPLES IN VCF!
      tmpGt <- tmpGt[,c(1,3,2)]
    }
  }
  info <- tmpVcf[,"INFO"]
  func <- sapply(info,function(x){
    temp <- unlist(strsplit(x,","))
    paste0(unique(na.omit(unlist(sapply(temp,function(x) unlist(strsplit(x,"|",fixed=TRUE))[2])))),collapse=",")
  })
  func <- unname(func)
  callers <- sapply(info,function(x){
    temp <- unlist(strsplit(x,";"))
    sub("CALLERS=","",temp[grepl("CALLERS=",temp)])
  })
  callers <- unname(callers)
  id <- paste0(tumorID,"-",tmpVcf[,1],":",tmpVcf[,2])
  if(length(colnames(tmpGt))==3){
    tmp <- data.frame(tmpVcf[,c(1,2,4,5,7)],tmpGt,tumorID,info,func,callers,id,stringsAsFactors=FALSE)
  } else {
    tmp <- data.frame(tmpVcf[,c(1,2,4,5,7)],tmpGt,normal=NA,tumorID,info,func,callers,id,stringsAsFactors=FALSE)
  }
  colnames(tmp) <- c("CHROM","POS","REF","ALT","FILTER","FORMAT","Tumor","Normal","Tumor_ID","snpEff","snpEff_func","caller","id")
  rownames(tmp) <- NULL
  return(tmp)
}

vcf <- lapply(filenames,function(x) vcfRead(x,vcfDir))
vcf <- do.call(rbind,vcf)
nrow(vcf) #306237

Warning message in dir.create("Analysis"):
“'Analysis' already exists”


In [4]:
# CORRECT FOR NA ALT
table(is.na(vcf$ALT))
# vcf <- vcf[!is.na(vcf$ALT),]

# CORRECT FOR MULTIPLE ALT
table(grepl(",",vcf$ALT))
tmp <- sapply(vcf$ALT,function(x) unlist(strsplit(x,",",fixed=TRUE))[1])
vcf$ALT <- tmp

# # CORRECT FOR NA LINES
# sum(is.na(vcf))


 FALSE 
306237 


 FALSE   TRUE 
302173   4064 

In [5]:
# SELECT CODING/SPLICING
tmp <- unique(vcf$snpEff_func)
tmp <- unique(unlist(sapply(tmp,function(x) unlist(strsplit(x,",")))))
tmp <- unique(unlist(sapply(tmp,function(x) unlist(strsplit(x,"&")))))
tmp <- tmp[!grepl("ENS",tmp)]
tmp <- tmp[!grepl("=",tmp)]
sort(tmp)
sel <- c("5_prime_UTR_premature_start_codon_gain_variant","5_prime_UTR_truncation", #NEW
         "3_prime_UTR_variant", "5_prime_UTR_variant", # Added
         "bidirectional_gene_fusion","conservative_inframe_deletion","conservative_inframe_insertion",
         "disruptive_inframe_deletion","disruptive_inframe_insertion",
         "exon_loss_variant","initiator_codon_variant","gene_fusion",
         "frameshift_variant","missense_variant","splice_acceptor_variant",
         "splice_donor_variant","splice_region_variant","start_lost","stop_gained",
         "stop_lost","stop_retained_variant")
setdiff(tmp,sel)
setdiff(sel,tmp)
sel <- sapply(sel,function(x) grep(x,vcf$snpEff_func))
sel <- unique(unlist(sel))
sel <- sort(sel)
sum(is.na(sel))
nrow(vcf) #227168
rownames(vcf) <- NULL
length(sel) #36938
vcf <- vcf[sel,]
sort(table(vcf$Tumor_ID))

callerN <- sapply(vcf$caller,function(x) length(unlist(strsplit(x,",",fixed=TRUE))))
callerN <- unname(callerN)
vcf <- data.frame(vcf,callerN,stringsAsFactors = FALSE)
table(vcf$callerN)
#   1     2     3     4     5     6 
# 29690  3753   704   328   438  2025  
rownames(vcf) <- NULL
#----------------------------------------------

save(vcf,file = "Analysis/vcf.rda")

[1] "3_prime_UTR_variant"                           
 [2] "5_prime_UTR_premature_start_codon_gain_variant"
 [3] "5_prime_UTR_truncation"                        
 [4] "5_prime_UTR_variant"                           
 [5] "bidirectional_gene_fusion"                     
 [6] "conservative_inframe_deletion"                 
 [7] "conservative_inframe_insertion"                
 [8] "disruptive_inframe_deletion"                   
 [9] "disruptive_inframe_insertion"                  
[10] "downstream_gene_variant"                       
[11] "exon_loss_variant"                             
[12] "frameshift_variant"                            
[13] "initiator_codon_variant"                       
[14] "intergenic_region"                             
[15] "intragenic_variant"                            
[16] "intron_variant"                                
[17] "missense_variant"                              
[18] "non_coding_transcript_exon_variant"            
[19] "non_coding_transcript_variant"                 
[20] "protein_protein_contact"                       
[21] "sequence_feature"                              
[22] "splice_acceptor_variant"                       
[23] "splice_donor_variant"                          
[24] "splice_region_variant"                         
[25] "start_lost"                                    
[26] "stop_gained"                                   
[27] "stop_lost"                                     
[28] "stop_retained_variant"                         
[29] "structural_interaction_variant"                
[30] "synonymous_variant"                            
[31] "TF_binding_site_variant"                       
[32] "TFBS_ablation"                                 
[33] "upstream_gene_variant"

[1] "downstream_gene_variant"            "intron_variant"                    
 [3] "non_coding_transcript_exon_variant" "upstream_gene_variant"             
 [5] "intergenic_region"                  "sequence_feature"                  
 [7] "TF_binding_site_variant"            "synonymous_variant"                
 [9] "intragenic_variant"                 "non_coding_transcript_variant"     
[11] "TFBS_ablation"                      "structural_interaction_variant"    
[13] "protein_protein_contact"

[1] "gene_fusion"

[1] 0

[1] 306237

[1] 36938


   AT4415     A1783     C0334 s3_7299_D     C8448     A9155 s1_3076_D s1_3295_D 
     1441      1545      1599      1692      1740      1767      1789      1807 
s2_6088_D     C0288     C1572    AT2550    AT2822 s2_8811_D s3_5832_D     A9878 
     1807      1813      1856      1880      1947      1958      1962      1993 
s3_7979_D s2_6339_D    AT4808 s1_8588_D 
     2073      2076      2082      2111 


    1     2     3     4     5     6 
29690  3753   704   328   438  2025 

Warning message in gzfile(file, "wb"):
“cannot open compressed file 'Analysis/vcf.rda', probable reason 'Permission denied'”


ERROR: Error in gzfile(file, "wb"): cannot open the connection


In [ ]:
# COUNTS and ALLELE FREQUENCY -----------------

countsVcf <- function(vcf,i){
  Tmp <- rep(NA,4)
  names(Tmp) <- c("t_ref_count","t_alt_count","n_ref_count","n_alt_count")
  vcfLine <- as.character(vcf[i,])
  names(vcfLine) <- colnames(vcf)
  caller <- unlist(strsplit(vcfLine["caller"],",",fixed=TRUE))[1]
  if(caller=="varscan"){
    tmp <- unlist(strsplit(vcfLine["Tumor"],":"))
    names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))
    Tmp["t_alt_count"] <- as.numeric(tmp["AD"])
    Tmp["t_ref_count"] <- as.numeric(tmp["RD"])
    tmp <- unlist(strsplit(vcfLine["Normal"],":"))
    names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))
    Tmp["n_alt_count"] <- as.numeric(tmp["AD"])
    Tmp["n_ref_count"] <- as.numeric(tmp["RD"])
  } else {
    tmp <- unlist(strsplit(vcfLine["Tumor"],":"))
    names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))
    if(tmp["AD"]!=".") Tmp[c("t_ref_count","t_alt_count")] <- as.numeric(unlist(strsplit(tmp["AD"],","))[1:2])
    if(!is.na(vcfLine["Normal"])) {
      tmp <- unlist(strsplit(vcfLine["Normal"],":"))
      names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))[1:length(tmp)]
      if(tmp["AD"]!=".") Tmp[c("n_ref_count","n_alt_count")] <- as.numeric(unlist(strsplit(tmp["AD"],","))[1:2])
    }
  }
  return(Tmp)
}

cl <- makePSOCKcluster(60,outfile="")
registerDoParallel(cl)
tmp <- foreach(i = 1:nrow(vcf)) %dopar% {
  tmp <- countsVcf(vcf,i)
  return(tmp)
}
stopCluster(cl)

tmp <- do.call(rbind,tmp)
dim(tmp) #36938     4

counts <- cbind(tmp[,1:2],NA,NA,tmp[,3:4],NA,NA)
colnames(counts) <- c("t_ref_count","t_alt_count","t_depth","t_vaf","n_ref_count","n_alt_count","n_depth","n_vaf")
counts[,"t_depth"] <- counts[,"t_alt_count"] + counts[,"t_ref_count"]
counts[,"t_vaf"] <- round(counts[,"t_alt_count"]/counts[,"t_depth"],2)
counts[,"n_depth"] <- counts[,"n_alt_count"] + counts[,"n_ref_count"]
counts[,"n_vaf"] <- round(counts[,"n_alt_count"]/counts[,"n_depth"],2)
rownames(counts) <- NULL

View(counts[sample(1:nrow(counts),100),])

vcf <- data.frame(vcf, counts, stringsAsFactors = FALSE)

table(cond <- is.na(vcf$n_depth)&!is.na(vcf$Normal))
# FALSE  TRUE 
# 27894  9044 
# table(cond <- is.na(vcf$n_depth))

tmp <- vcf[cond,]

# table(tmp$caller)
# freebayes 
# 9044

vcf <- vcf[!cond,]

rownames(vcf) <- NULL
nrow(vcf) #27894
rm(counts)

#----------------------------------------------
# save(vcf,file = "Analysis/vcf.rda")